Goal: Analyze duplicate matrix of questions.

So far we've been focusing efforts on just looking at question pairs. But the question IDs also give some useful information, an undirected graph between questions. Let's see if we can extract some information from them.

In [1]:
import pandas as pd
# Make my plots pretty!
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['figure.dpi'] = 100

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [104]:
# Read training data.
train = pd.read_csv('../data/train.csv')

In [3]:
train[:10]

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [91]:
qid1 = train['qid1']
qid2 = train['qid2']
qid1_unique = qid1.unique()
qid2_unique = qid2.unique()
print('Q1 unique: {0}/{2} ({1}%)'.format(len(qid1_unique), len(qid1_unique) * 100.0 / len(qid1), len(qid1)))
print('Q2 unique: {0}/{2} ({1}%)'.format(len(qid1_unique), len(qid1_unique) * 100.0 / len(qid1), len(qid2)))
qid1_qid2 = set(list(qid1_unique) + list(qid2_unique))
print('Q1 U Q2 unique: {0}/{2} ({1}%)'.format(len(qid1_qid2), len(qid1_qid2) * 100.0 / len(qid1), len(qid1)))

Q1 unique: 290654/404290 (71.8924534369%)
Q2 unique: 290654/404290 (71.8924534369%)
Q1 U Q2 unique: 537933/404290 (133.056222019%)


In [112]:
dups = train[train['is_duplicate'] == 1][['qid1', 'qid2']]
len(dups)

149263

In [107]:
train[['qid1']].to_csv('qid1.txt', header=False, index=False)
!tail qid1.txt

537922
99131
1931
537924
537926
433578
18840
537928
537930
537932


In [108]:
train[['qid2']].to_csv('qid2.txt', header=False, index=False)
!tail qid2.txt

537923
81495
16773
537925
537927
379845
155606
537929
537931
537933


In [109]:
!cat qid1.txt qid2.txt | sort -n | uniq > qids.txt

In [110]:
!wc -l qids.txt

  537933 qids.txt


In [115]:
dups.to_csv('dups0.csv', header=False, index=False)
!tail dups0.csv

25994,16064
20171,290649
128018,14005
537915,537916
178643,87385
537922,537923
99131,81495
1931,16773
537926,537927
18840,155606


In [119]:
dups[['qid2', 'qid1']].to_csv('dups1.csv', header=False, index=False)
!tail dups1.csv

16064,25994
290649,20171
14005,128018
537916,537915
87385,178643
537923,537922
81495,99131
16773,1931
537927,537926
155606,18840


In [133]:
!cat qids.txt | sed 's/\(.*\)/\1,\1/' > dups2.csv

In [134]:
!cat dups0.csv dups1.csv dups2.csv | sort -g | uniq > dups.csv

In [135]:
!wc -l dups.csv

  836459 dups.csv


# After Getting Results

Used a mapreduce job https://github.com/jfkelley/hadoop-matrix-mult to get the transitive closure (i.e. link up all the duplicate chains)

From dumbo:

In [139]:
!tail ../data/all_duplicates.tsv

116636	116636	3.0
122039	122039	2.0
275072	275072	1.0
114203	114203	1.0
338360	338360	1.0
477209	477209	1.0
80153	17456	109.0
260825	101894	3.0
290732	290732	1.0
270374	270374	1.0


In [138]:
qs[qs['qid1'] == 80153]

,qid1,question1
41380,80153,How should I loose weight?


I don't know what I expected.

In [142]:
alldups = pd.read_csv('../data/all_duplicates.tsv', header=None, index_col=None, sep='\t')

In [143]:
alldups.columns = ['qid1', 'qid2', 'ignore']

In [197]:
dupset = set((t.qid1, t.qid2) for t in alldups.itertuples())

In [153]:
# How many dups does q1 have
qcounts = alldups.groupby('qid1').count()['qid2'].sort_values()

In [154]:
qcounts.describe()

count    537933.000000
mean          1.849727
std           4.182115
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         109.000000
Name: qid2, dtype: float64

In [240]:
# Sentences with the most duplicates
dups = list(qcounts[-500:].index)

In [241]:
def sampler(sz=10):
    np.random.shuffle(dups)
    d = dups[:sz]
    print(d)
    sample = np.zeros((sz, sz))
    for i, x in enumerate(d):
        for j, y in enumerate(d[i+1:]):
            if (x, y) in dupset:
                sample[i,j] = 1
    return sample

In [242]:
(80153, 17456) in dupset

True

In [243]:
sampler(10)

[57359, 19329, 69550, 42615, 80153, 17258, 178157, 13748, 72620, 62742]


array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [237]:
len(qcounts[qcounts < 2])

388283

In [238]:
len(alldups)

995029

In [239]:
len(qcounts[qcounts < 2]) * 1.0 / len(alldups)

0.39022279752650424

Basically, 380k questions (40%) don't have duplicates in the database. The duplicate graphs are also a tiny part of the entire database. Only 1 out of 1 million possible pairs are duplicates.